In [2]:
# Activate intractive functionality of matplotlib
%matplotlib ipympl
# Activate auto reload 
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import libraries
import os
import os.path as path
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from faker import Factory
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 
import itertools 
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from ipywidgets import HBox, VBox
from ipywidgets import IntSlider, Output
from IPython.display import clear_output
import matplotlib.colors as mcolors
import h5py
import re
import matplotlib
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

# Local module and scripts
from pyccapt.calibration.calibration_tools import tools, share_variables, calibration, ion_selection
from pyccapt.calibration.calibration_tools import widgets as wd
from pyccapt.calibration.data_tools import data_tools, data_loadcrop, dataset_path_qt
from pyccapt.calibration.mc import mc_tools
from pyccapt.calibration.leap_tools import ccapt_tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
button = widgets.Button(
    description='load dataset',
)
@button.on_click
def open_file_on_click(b):
    """
    Event handler for button click event.
    Prompts the user to select a dataset file and stores the selected file path in the global variable dataset_path.
    """
    global dataset_path
    dataset_path = dataset_path_qt.gui_fname().decode('ASCII')
button

Button(description='load dataset', style=ButtonStyle())

In [ ]:
tdc, pulse_mode, flightPathLength_d, t0_d, max_mc, det_diam = wd.dataset_instrument_specification_selection()
display(tdc, flightPathLength_d)

$$\textbf{You can specify which dataset to use in below block}$$

In [19]:
# exctract needed data from Pandas data frame as an numpy array
# create an instance of the Variables opject
variables = share_variables.Variables()
dataset_main_path = os.path.dirname(dataset_path)
dataset_name_with_extention = os.path.basename(dataset_path)
dataset_main_path = os.path.dirname(dataset_main_path)
variables.dataset_name = os.path.splitext(dataset_name_with_extention)[0]
variables.result_data_path = dataset_main_path + '/ions_selection/'
variables.result_data_name = 'range_' + variables.dataset_name
variables.result_path = dataset_main_path + '/' + '/ions_selection/'

if not os.path.isdir(variables.result_path):
    os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
    
# Create data farame out of hdf5 file dataset
data = data_tools.load_data(dataset_path, tdc.value, mode='processed')
# extract data from the path and create the Variable object
data_tools.extract_data(data, variables, flightPathLength_d.value, max_mc.value)

The maximum time of flight: 5010


In [20]:
print('The data will be saved on the path:', variables.result_data_path)
print('=============================')
print('The dataset name after saving is:', variables.result_data_name)
print('=============================')
print('The figures will be saved on the path:', variables.result_path)
print('=============================')
print('Total number of Ions:', len(data))
data

The data will be saved on the path: D:/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4/ions_selection/
The dataset name after saving is: range_pyccapt_data_1642_Aug-30-2023_16-05_Al_test4
The figures will be saved on the path: D:/pyccapt/tests/data/data_1642_Aug-30-2023_16-05_Al_test4//ions_selection/
Total number of Ions: 10559766


,x (nm),y (nm),z (nm),mc_c (Da),mc (Da),high_voltage (V),pulse,start_counter,t_c (ns),t (ns),x_det (cm),y_det (cm),pulse_pi,ion_pp
0,21.387474,4.793744,3.742590,27.362214,28.993453,5019.959961,1003.992004,7565,572.395831,618.564148,2.651428,0.594286,0,0
1,-0.465509,5.449197,0.226819,27.034338,29.360520,5019.959961,1003.992004,7664,569.176119,606.260925,-0.055510,0.649796,99,2
2,-0.903869,5.094536,0.202985,27.100974,29.441372,5019.959961,1003.992004,7682,569.832045,606.932983,-0.107755,0.607347,18,1
3,18.634826,-3.726965,2.792876,27.100149,28.725689,5019.959961,1003.992004,7690,569.823932,611.520996,2.285714,-0.457143,8,1
4,4.014661,14.361170,1.705268,27.236032,29.512555,5019.959961,1003.992004,7794,571.159024,614.298462,0.486531,1.740408,104,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10559761,-4.831920,15.283945,57.205555,27.244832,29.759238,6351.169922,1270.234009,19549,571.245368,550.999146,-0.460408,1.456326,105,1
10559762,-14.125940,-1.271678,56.866164,27.414528,29.967560,6351.169922,1270.234009,19664,572.907770,551.705505,-1.342041,-0.120816,31,1
10559763,16.429191,3.148928,57.345214,27.119032,29.029561,6351.169922,1270.234009,19762,570.009661,545.156128,1.567347,0.300408,98,1
10559764,13.831981,-9.107195,57.311214,26.980992,28.815053,6351.169922,1270.234009,19833,568.650437,543.187866,1.319184,-0.868571,71,1


In [21]:
interact_manual(tools.mc_hist_plot, variables=fixed(variables), bin_size=widgets.FloatText(value=0.1), 
                mode=widgets.Dropdown(options=[('mc_c', 'mc_c'), ('tof_c', 'tof_c')]), prominence=widgets.IntText(value=50), distance=widgets.IntText(value=1), 
                lim=widgets.IntText(value=400), percent=widgets.IntText(value=50), selector=fixed('None'), plot=fixed(True), figname=widgets.Text(value='hist'),
                peaks_find_plot=widgets.Dropdown(options=[('True', True), ('False', False)]));

interactive(children=(FloatText(value=0.1, description='bin_size'), Dropdown(description='mode', options=(('mc…

In [22]:
bin_size=widgets.FloatText(value=0.1, description='bin size:')
prominence=widgets.IntText(value=50, description='peak prominance:')
distance=widgets.IntText(value=1, description='peak distance:')
lim_tof=widgets.IntText(value=400, description='lim tof/mc:')
percent=widgets.IntText(value=50, description='percent MRP:')
index_fig = widgets.IntText(value=1, description='fig index:')
plot_peak = widgets.Dropdown(
    options=[('True', True), ('False', False)],
    description='plot peak:'
)

def hist_plot_p(variables, plot):
    
    with out:
        clear_output(True)
        # clear the peak_idx
        variables.peaks_idx = []
        bin_size_p = bin_size.value
        prominence_p = prominence.value
        distance_p = distance.value
        lim_tof_p = lim_tof.value
        percent_p = percent.value
        index_fig_p = index_fig.value
        plot_peak_p = plot_peak.value
        tools.hist_plot(variables.mc_c[variables.mc_c < lim_tof_p], variables, bin_size_p, label='mc', range_data=None, adjust_label=False, ranging=False,
              log=True, mode='count', percent=percent_p, peaks_find=True, peaks_find_plot=plot_peak_p, plot=True, prominence=prominence_p,
              distance=None, h_line=False, selector='peak', fast_hist=True, fig_name=index_fig_p, text_loc='right',
              fig_size=(9, 5), background={'calculation': False})

def hist_plot_r(variables, plot):
    
    with out:
        clear_output(True)
        bin_size_r = bin_size.value
        prominence_r = prominence.value
        distance_r = distance.value
        lim_tof_r = lim_tof.value
        percent_r = percent.value
        index_fig_r = index_fig.value
        plot_peak_r = plot_peak.value
        print('=============================')
        print('Press left click to draw a line')
        print('Press right click to remove a line')
        print('Press r to remove all the line')
        print('Hold shift and use mouse scroll for zooming on x axis')
        print('Hold ctrl and left mouse bottom to move a line')
        print('=============================')
        tools.hist_plot(variables.mc_c[variables.mc_c < lim_tof_r], variables, bin_size_r, label='mc', range_data=None, adjust_label=False, ranging=False,
              log=True, mode='count', percent=percent_r, peaks_find=False, peaks_find_plot=plot_peak_r, plot=True, prominence=prominence_r,
              distance=None, h_line=False, selector='range', fast_hist=True, fig_name=index_fig_r, text_loc='right',
              fig_size=(9, 5), background={'calculation': False})

In [23]:
# element calculate
peak_val = widgets.FloatText(value=1.1, description='peak value:')
charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)
aboundance_threshold = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)
num_element = widgets.IntText(value=5, description='num element:')
# formula calculate
formula_m = widgets.Text(
    value='{12}C1{16}O2',
    placeholder='Type a formula  {12}C1{16}O2',
    description='Isotope formula:',
    disabled=False
)

molecule_charge = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)

aboundance_threshold_molecule = widgets.FloatText(value=0.0, description='threshold aboundance:', min=0, max=1, step=0.1)

# molecule create
formula_com = widgets.Text(
    value='',
    placeholder="H, O",
    description='Elements:',
    disabled=False
)
complexity = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='complexity:'
)

charge_com = widgets.Dropdown(
    options=[('1', 1), ('2', 2), ('3', 3), ('4', 4),('5', 5), ('6', 6)],
    value=3,
    description='charge:'
)

In [28]:
plot_button_p = widgets.Button(
    description='plot hist',
)

plot_button = widgets.Button(
    description='plot hist',
)


find_elem_button = widgets.Button(
    description='find element',
)
find_mole_button = widgets.Button(
    description='find molecule',
)

formula_button = widgets.Button(
    description='manual formula',
)

add_ion_button = widgets.Button(
    description='add ion',
)
romove_ion_button = widgets.Button(
    description='remove ion',
)
show_color = widgets.Button(
    description='show color',
)
change_color = widgets.Button(
    description='change color',
)

color_picker = widgets.ColorPicker(description='Select a color:')
row_index = widgets.IntText(value=0, description='index row:')
mass_unknown = widgets.FloatText(value=0.0, description='unknown mass:')
@plot_button_p.on_click
def plot_on_click_p(b, plot=True):
    hist_plot_p(variables, plot)
    
    
@find_elem_button.on_click
def vol_on_click(b,):
    with output2:
        peak_val_s = peak_val.value
        charge_s = charge.value
        num_element_s = num_element.value
        aboundance_threshold_s = aboundance_threshold.value
        clear_output(True)
        df = ion_selection.find_closest_elements(peak_val_s, num_element_s, aboundance_threshold_s, charge=charge_s, variables=variables)
        display(df)
    
@find_mole_button.on_click
def mol_on_click(b,):
    with output2:
        if formula_com.value == '':
            print("Input is empty. Type at least two elements.")
        else:
            df = ion_selection.molecule_create(formula_com.value, complexity.value, charge_com.value, aboundance_threshold_molecule.value, variables)
            clear_output(True)
            display(df)
    
@formula_button.on_click
def manual_formula(b,):
    with output2:
        if formula_m.value == '':
            print("Input is empty. Type the formula.")
        else:
            df = ion_selection.molecule_manual(formula_m.value, molecule_charge.value, latex=True, variables=variables)
            clear_output(True)
            display(df)

@add_ion_button.on_click
def add_ion_to_range_dataset(b,):
    ion_selection.ranging_dataset_create(variables, row_index.value, mass_unknown.value)
    with output3:
        clear_output(True)
        display(variables.range_data)
@romove_ion_button.on_click
def remove_ion_to_range_dataset(b,):
    if len(variables.range_data) >= 1:
        variables.range_data = variables.range_data.drop(len(variables.range_data) - 1)
        with output3:
            clear_output(True)
            display(variables.range_data)
        
@show_color.on_click
def show_color_ions(b,):
    with output3:
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))
@change_color.on_click
def change_color_m(b,):
    with output3:
        selected_color = mcolors.to_hex(color_picker.value)
        variables.range_data.at[row_index.value, 'color'] = selected_color
        clear_output(True)
        display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

        # Create "Next" and "Previous" buttons
start_button = widgets.Button(description="Start")
next_button = widgets.Button(description="Next")
prev_button = widgets.Button(description="Previous")
reset_h_line_button = widgets.Button(description="Reset")
all_peaks_button = widgets.Button(description="Add all peaks")


# Define button click events
@start_button.on_click
def start_peak(_):
    variables.h_line_pos = []
    print('=============================')
    print('Press left click to draw a line')
    print('Press right click to remove a line')
    print('Press r to remove all the line')
    print('Press a to automatically draw lines')
    print('Hold shift and use mouse scroll for zooming on x axis')
    print('Hold ctrl and left mouse bottom to move a line')
    print('=============================')
    variables.peaks_index = 0
    peak = variables.peak[variables.peaks_idx[variables.peaks_index]]
    peak_val.value = peak
    print('peak idc:', variables.peaks_index, 'Peak location:', peak)
    mask = (variables.mc_c < peak + 5) & (variables.mc_c > peak - 5)
    mc = np.copy(variables.mc_calib)
    peaks = np.copy(variables.peak)
    variables.mc_c = variables.mc_c[mask]
    hist_plot_r(variables, plot=True)
    variables.mc_c = mc
    variables.peak = peaks
@next_button.on_click
def next_peak(_):
    variables.h_line_pos = []
    variables.peaks_index = variables.peaks_index + 1
    peak = variables.peak[variables.peaks_idx[variables.peaks_index]]
    peak_val.value = peak
    print('peak idc:', variables.peaks_index, 'Peak location:', peak)
    mask = (variables.mc_c < peak + 5) & (variables.mc_c > peak - 5)
    mc = np.copy(variables.mc_calib)
    peaks = np.copy(variables.peak)
    variables.mc_c = variables.mc_c[mask]
    hist_plot_r(variables, plot=True)
    variables.mc_c = mc
    variables.peak = peaks
@prev_button.on_click
def prev_peak(_):
    variables.h_line_pos = []
    variables.peaks_index = variables.peaks_index - 1
    peak = variables.peak[variables.peaks_idx[variables.peaks_index]]
    peak_val.value = peak
    print('peak idc:', variables.peaks_index, 'Peak location:', peak)
    mask = (variables.mc_c < peak + 5) & (variables.mc_c > peak - 5)
    mc = np.copy(variables.mc_calib)
    peaks = np.copy(variables.peak)
    variables.mc_c = variables.mc_c[mask] 
    hist_plot_r(variables, plot=True)
    variables.mc_c = mc
    variables.peak = peaks

@reset_h_line_button.on_click
def rest_h_line(_):
    variables.h_line_pos = []
@all_peaks_button.on_click
def select_all_peaks(_):
    variables.peaks_idx = list(range(0,len(variables.peak)))
    
tab1 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, plot_peak, HBox(children=[plot_button_p, all_peaks_button])])
tab2 = VBox(children=[bin_size, index_fig, prominence, distance, lim_tof, percent, plot_peak, HBox(children=[VBox(children=[start_button, next_button, prev_button, reset_h_line_button])])])
tab4 = VBox(children=[HBox(children=[VBox(children=[peak_val, num_element, charge, aboundance_threshold, 
                      find_elem_button]),  VBox(children=[formula_com, complexity, charge_com, aboundance_threshold_molecule, find_mole_button]),
                      VBox(children=[formula_m, molecule_charge, formula_button])]), VBox(children=[row_index, color_picker, mass_unknown, add_ion_button, romove_ion_button, 
                                                                                                                                      show_color, change_color])])

tabs1 = widgets.Tab(children=[tab1, tab2])
tabs2 = widgets.Tab(children=[tab4])
tabs1.set_title(0, 'peak finder')
tabs1.set_title(1, 'rangging')
tabs2.set_title(0, 'element finder')
# Create two Output widgets to capture the output of each plot
out = Output()
output2 = Output()
output3 = Output()

# Create an HBox to display the buttons side by side
buttons_layout = widgets.HBox([tabs1, tabs2])

# Create a VBox to display the output widgets below the buttons
output_layout_h = widgets.HBox([out, output3])
output_layout = widgets.VBox([output_layout_h, output2])


# Display the buttons and the output widgets
display(buttons_layout, output_layout)


with output3:
    display(variables.range_data)

In [25]:
def mc_hist_plot(bin_size, lim_mc, hist_color_range, figname):
    peaks_ini, peaks_y_ini, peak_widths_p_ini, _ = tools.hist_plot(variables.mc_c[variables.mc_c < lim_mc], variables, bin=bin_size, range_data=variables.range_data, ranging=True, hist_color_range=hist_color_range, adjust_label=False, peaks_find=True, peaks_find_plot=True, plot=True, label='mc', fig_name=figname)
   
interact_manual(mc_hist_plot, bin_size=widgets.FloatText(value=0.1), lim_mc=widgets.IntText(value=400), hist_color_range=widgets.Dropdown(options=[('True', True), ('False', False)])
                , figname=widgets.Text(value='ranged'));

interactive(children=(FloatText(value=0.1, description='bin_size'), IntText(value=400, description='lim_mc'), …

In [15]:
display(variables.range_data.style.applymap(ion_selection.display_color, subset=['color']))

,ion,mass,mc,mc_low,mc_up,color,peak_count,element,complex,isotope,charge
0,${}^{1}H^{+}$,1.010000,1.002547,0.939734,1.201835,#b2aa2d,2615,H,1,1,1
1,${}^{27}Al^{2+}$,13.490000,13.433484,13.878650,15.597560,#e7e0d1,291781,Al,1,27,2
2,${}^{27}Al^{+}$,26.980000,26.967166,27.838116,31.591418,#e7e0d1,3606413,Al,1,27,1


In [16]:
variables.range_data.dtypes

ion            object
mass          float64
mc            float64
mc_low        float64
mc_up         float64
color          object
peak_count      int64
element        object
complex         int32
isotope         int64
charge          int32
dtype: object

In [17]:
# save the new data
name_save_file = variables.result_data_path + '/' + variables.result_data_name + '.h5'
data_tools.store_df_to_hdf(variables.range_data,  'df', name_save_file)

In [18]:
# save data in csv format
name_save_file = variables.result_data_path + '/' + variables.result_data_name + '.csv'
data_tools.store_df_to_csv(variables.range_data, name_save_file)